In [8]:
# %%
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Layer
import tensorflow as tf


In [9]:
# Configuration parameters
PERIOD_SIZE = 50
STEP_SIZE = 100
MEDIAN_Q_THRESHOLD = 0.5
LOWER_Q_THRESHOLD = 0.1
UPPER_Q_THRESHOLD = 0.9
ANOMALY_RES = [15, 11]  # Ground truth anomaly values
RANDOM_SEEDS = [42, 123]  # For ensemble diversity
LOOK_BACK = PERIOD_SIZE
TW = STEP_SIZE
MULTI = LOOK_BACK // TW


In [10]:
# Custom Swish Activation (CORRECTED)
class Swish(Layer):
    def __init__(self, beta=1.0, **kwargs):
        super(Swish, self).__init__(**kwargs)
        self.beta = K.cast_to_floatx(beta)

    def call(self, inputs):
        # Correct Swish formula: x * sigmoid(beta * x)
        return inputs * K.sigmoid(self.beta * inputs)

    def get_config(self):
        config = {'beta': float(self.beta)}
        base_config = super(Swish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

# %%
# Utility Functions
def verify_stationarity(dataset):
    """Check if time series is stationary using ADF test"""
    is_stationary = True
    test_results = adfuller(dataset)

    print(f"ADF test statistic: {test_results[0]:.4f}")
    print(f"p-value: {test_results[1]:.4f}")
    print("Critical thresholds:")

    critical_value = None
    for i, (key, value) in enumerate(test_results[4].items()):
        print(f"\t{key}: {value:.3f}")
        if i == 0:
            critical_value = value

    if test_results[0] > critical_value:
        print('Series is non-stationary')
        is_stationary = False
    else:
        print('Series is stationary')
    
    return is_stationary


def create_dataset(dataset, look_back=1, tw=3):
    """Create quantile-based features for training"""
    dataX, dataY = [], []  # q50 (median)
    dataUpperX, dataUpperY = [], []  # q90 (upper)
    dataLowerX, dataLowerY = [], []  # q10 (lower)
    multi = look_back // tw
    
    for i in range(len(dataset) - look_back - 1):
        q50X, q90X, q10X = [], [], []
        a = dataset[i + 1:(i + look_back + 1)]
        
        # Calculate target quantiles
        c = np.quantile(a, MEDIAN_Q_THRESHOLD)
        u = np.quantile(a, UPPER_Q_THRESHOLD)
        l = np.quantile(a, LOWER_Q_THRESHOLD)
        
        # Create sub-window features
        for j in range(0, len(a), tw):
            window = a[j:j + tw]
            q50X.append(np.quantile(window, MEDIAN_Q_THRESHOLD))
            q90X.append(np.quantile(window, UPPER_Q_THRESHOLD))
            q10X.append(np.quantile(window, LOWER_Q_THRESHOLD))
        
        dataX.append(q50X)
        dataY.append(c)
        dataUpperX.append(q90X)
        dataUpperY.append(u)
        dataLowerX.append(q10X)
        dataLowerY.append(l)
    
    return (np.array(dataX), np.array(dataY), 
            np.array(dataUpperX), np.array(dataUpperY), 
            np.array(dataLowerX), np.array(dataLowerY))


def quantile_loss(q):
    """Proper quantile loss function for training"""
    def loss(y_true, y_pred):
        e = y_true - y_pred
        return K.mean(K.maximum(q * e, (q - 1) * e))
    return loss


def create_quantile_model(quantile, input_shape, seed=42, units=64):
    """Create improved LSTM model with proper architecture"""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    model = Sequential([
        LSTM(units, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units // 2),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1),
        Swish(beta=1.5)  # Single activation layer
    ])
    
    model.compile(
        loss=quantile_loss(quantile),
        optimizer='adam',
        metrics=['mse']
    )
    
    return model


def predict_ensemble(models, input_data):
    """Average predictions from multiple models"""
    predictions = [model.predict(input_data, verbose=0) for model in models]
    return np.mean(predictions, axis=0)


def identify_alpha(dataset):
    """Calculate trend stability metric"""
    alpha_detection = []
    prev_slope = 1
    
    for m in range(0, len(dataset), PERIOD_SIZE):
        period_dataset = dataset[m:m + PERIOD_SIZE]
        if len(period_dataset) < 2:
            continue
        # Extract scalar values from arrays
        start_val = float(period_dataset[0][0]) if period_dataset[0].ndim > 0 else float(period_dataset[0])
        end_val = float(period_dataset[-1][0]) if period_dataset[-1].ndim > 0 else float(period_dataset[-1])
        slope = (end_val - start_val) / len(period_dataset)
        alpha = slope / prev_slope if prev_slope != 0 else 1
        alpha_detection.append(float(alpha))
        prev_slope = slope if slope != 0 else 1
    
    return float(np.abs(np.mean(alpha_detection))) if alpha_detection else 1.0


LOADING AND PREPROCESSING DATA

In [11]:

np.random.seed(7)
dataframe = read_csv('Q-TV-RNN/Q-data/speed_t4013_train.csv', usecols=[2], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

# Check for NaN/Inf
assert not np.isnan(dataset).any(), "Dataset contains NaN values"
assert not np.isinf(dataset).any(), "Dataset contains Inf values"

print(f"\nDataset shape: {dataset.shape}")
print(f"Dataset range: [{dataset.min():.2f}, {dataset.max():.2f}]")

# Verify stationarity
print("\nStationarity Test:")
stationary = verify_stationarity(dataset)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# Split into train/validation/test sets (60/20/20)
train_size = int(len(dataset) * 0.6)
val_size = int(len(dataset) * 0.2)
train = dataset[0:train_size, :]
val = dataset[train_size:train_size + val_size, :]
test = dataset[train_size + val_size:, :]

print(f"\nTrain size: {len(train)}")
print(f"Validation size: {len(val)}")
print(f"Test size: {len(test)}")

# Calculate alpha (trend metric)
alpha = identify_alpha(dataset)
print(f"\nAlpha (trend stability): {alpha:.4f}")

# %%
# Create datasets
print("\n" + "="*60)
print("CREATING QUANTILE DATASETS")
print("="*60)

trainX, trainY, trainXU, trainYU, trainXL, trainYL = create_dataset(train, LOOK_BACK, TW)
valX, valY, valXU, valYU, valXL, valYL = create_dataset(val, LOOK_BACK, TW)
testX, testY, testXU, testYU, testXL, testYL = create_dataset(test, LOOK_BACK, TW)

# Reshape for LSTM [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
valX = np.reshape(valX, (valX.shape[0], valX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

trainXU = np.reshape(trainXU, (trainXU.shape[0], trainXU.shape[1], 1))
valXU = np.reshape(valXU, (valXU.shape[0], valXU.shape[1], 1))
testXU = np.reshape(testXU, (testXU.shape[0], testXU.shape[1], 1))

trainXL = np.reshape(trainXL, (trainXL.shape[0], trainXL.shape[1], 1))
valXL = np.reshape(valXL, (valXL.shape[0], valXL.shape[1], 1))
testXL = np.reshape(testXL, (testXL.shape[0], testXL.shape[1], 1))



Dataset shape: (1745, 1)
Dataset range: [17.00, 77.00]

Stationarity Test:
ADF test statistic: -22.0681
p-value: 0.0000
Critical thresholds:
	1%: -3.434
	5%: -2.863
	10%: -2.568
Series is stationary

Train size: 1047
Validation size: 349
Test size: 349

Alpha (trend stability): 1.1422

CREATING QUANTILE DATASETS


In [12]:

input_shape = (MULTI, 1)
epochs = 100
batch_size = 32

# Callbacks
callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, verbose=1)
]


In [13]:
models_q50 = []
for i, seed in enumerate(RANDOM_SEEDS):
    print(f"\nModel {i+1}/3 (seed={seed}):")
    model = create_quantile_model(MEDIAN_Q_THRESHOLD, input_shape, seed)
    history = model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=0
    )
    models_q50.append(model)
    print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
    print(f"  Final val loss: {history.history['val_loss'][-1]:.6f}")

# Train Q10 (lower) models
print("\n--- Training Q10 (Lower Bound) Models ---")
models_q10 = []
for i, seed in enumerate(RANDOM_SEEDS):
    print(f"\nModel {i+1}/3 (seed={seed}):")
    model = create_quantile_model(LOWER_Q_THRESHOLD, input_shape, seed)
    history = model.fit(
        trainXL, trainYL,
        validation_data=(valXL, valYL),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=0
    )
    models_q10.append(model)
    print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
    print(f"  Final val loss: {history.history['val_loss'][-1]:.6f}")

# Train Q90 (upper) models
print("\n--- Training Q90 (Upper Bound) Models ---")
models_q90 = []
for i, seed in enumerate(RANDOM_SEEDS):
    print(f"\nModel {i+1}/3 (seed={seed}):")
    model = create_quantile_model(UPPER_Q_THRESHOLD, input_shape, seed)
    history = model.fit(
        trainXU, trainYU,
        validation_data=(valXU, valYU),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=0
    )
    models_q90.append(model)
    print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
    print(f"  Final val loss: {history.history['val_loss'][-1]:.6f}")

print("\n✓ All ensemble models trained successfully!")

# %%
# Determine optimal threshold from validation set
print("\n" + "="*60)
print("LEARNING OPTIMAL THRESHOLD FROM VALIDATION SET")
print("="*60)

val_predictions_q50 = predict_ensemble(models_q50, valX)
val_predictions_q10 = predict_ensemble(models_q10, valXL)
val_predictions_q90 = predict_ensemble(models_q90, valXU)

val_errors = []
for i in range(len(valX)):
    actual = val[LOOK_BACK + i + 1]
    iqr = val_predictions_q90[i] - val_predictions_q10[i]
    error = abs(actual - val_predictions_q50[i]) / (iqr + 1e-6)
    val_errors.append(error[0])

# Use 95th percentile of validation errors as threshold
threshold_multiplier = np.quantile(val_errors, 0.95)
print(f"Learned threshold multiplier: {threshold_multiplier:.4f}")



Model 1/3 (seed=42):


ValueError: Exception encountered when calling layer "lstm_4" (type LSTM).

slice index 0 of dimension 0 out of bounds. for '{{node strided_slice_1}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1](transpose, strided_slice_1/stack, strided_slice_1/stack_1, strided_slice_1/stack_2)' with input shapes: [0,?,1], [1], [1], [1] and with computed input tensors: input[1] = <0>, input[2] = <1>, input[3] = <1>.

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 0, 1), dtype=float32)
  • mask=None
  • training=None
  • initial_state=None